# 2D Materials ML - Exploratory Data Analysis

This notebook provides exploratory analysis for 2D materials data, focusing on band gap and formation energy prediction.

## 1. Setup and Import Libraries

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

## 2. Load Data

In [ ]:
# Define paths
data_dir = Path('../data/processed')
raw_dir = Path('../data/raw')

# Load processed data
processed_file = data_dir / 'materials_processed.csv'
if processed_file.exists():
    df = pd.read_csv(processed_file)
    print(f"Loaded {len(df)} materials from {processed_file}")
else:
    print("No processed data found. Run 'python main.py' first!")
    df = pd.DataFrame()

In [ ]:
# Display basic information
if not df.empty:
    print(f"Dataset shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nFirst 5 rows:")
    display(df.head())

## 3. Data Overview

In [ ]:
# Data types and missing values
if not df.empty:
    print("Data Info:")
    df.info()
    
    print("\nMissing Values:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percentage': missing_pct
    })
    display(missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False))

In [ ]:
# Statistical summary
if not df.empty:
    print("Statistical Summary:")
    display(df.describe())

## 4. Target Variable Analysis

In [ ]:
# Band gap analysis
if 'band_gap' in df.columns:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Distribution
    axes[0].hist(df['band_gap'].dropna(), bins=30, edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Band Gap (eV)')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Band Gap Distribution')
    axes[0].axvline(df['band_gap'].mean(), color='red', linestyle='--', label=f'Mean: {df["band_gap"].mean():.2f}')
    axes[0].legend()
    
    # Box plot
    axes[1].boxplot(df['band_gap'].dropna())
    axes[1].set_ylabel('Band Gap (eV)')
    axes[1].set_title('Band Gap Box Plot')
    
    # Q-Q plot
    from scipy import stats
    stats.probplot(df['band_gap'].dropna(), dist="norm", plot=axes[2])
    axes[2].set_title('Band Gap Q-Q Plot')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Band Gap Statistics:")
    print(df['band_gap'].describe())

In [ ]:
# Formation energy analysis
if 'formation_energy' in df.columns:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Distribution
    axes[0].hist(df['formation_energy'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='green')
    axes[0].set_xlabel('Formation Energy (eV/atom)')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Formation Energy Distribution')
    axes[0].axvline(df['formation_energy'].mean(), color='red', linestyle='--', 
                   label=f'Mean: {df["formation_energy"].mean():.2f}')
    axes[0].legend()
    
    # Box plot
    axes[1].boxplot(df['formation_energy'].dropna())
    axes[1].set_ylabel('Formation Energy (eV/atom)')
    axes[1].set_title('Formation Energy Box Plot')
    
    # Q-Q plot
    stats.probplot(df['formation_energy'].dropna(), dist="norm", plot=axes[2])
    axes[2].set_title('Formation Energy Q-Q Plot')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Formation Energy Statistics:")
    print(df['formation_energy'].describe())

## 5. Feature Correlations

In [ ]:
# Correlation matrix
numeric_cols = df.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 1:
    corr_matrix = df[numeric_cols].corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # Most correlated features with targets
    if 'band_gap' in corr_matrix.columns:
        print("Top correlations with Band Gap:")
        bg_corr = corr_matrix['band_gap'].abs().sort_values(ascending=False)[1:6]
        print(bg_corr)
    
    if 'formation_energy' in corr_matrix.columns:
        print("\nTop correlations with Formation Energy:")
        fe_corr = corr_matrix['formation_energy'].abs().sort_values(ascending=False)[1:6]
        print(fe_corr)

## 6. Relationship Analysis

In [ ]:
# Band gap vs Formation energy
if all(col in df.columns for col in ['band_gap', 'formation_energy']):
    valid_data = df[['band_gap', 'formation_energy']].dropna()
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    scatter = ax.scatter(valid_data['formation_energy'], valid_data['band_gap'],
                        alpha=0.6, s=30, c=valid_data['band_gap'], cmap='viridis')
    
    ax.set_xlabel('Formation Energy (eV/atom)', fontsize=12)
    ax.set_ylabel('Band Gap (eV)', fontsize=12)
    ax.set_title('Band Gap vs Formation Energy', fontsize=14)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label('Band Gap (eV)', fontsize=10)
    
    # Add trend line
    z = np.polyfit(valid_data['formation_energy'], valid_data['band_gap'], 1)
    p = np.poly1d(z)
    ax.plot(valid_data['formation_energy'].sort_values(), 
            p(valid_data['formation_energy'].sort_values()),
            "r--", alpha=0.8, label=f'Trend: y={z[0]:.2f}x+{z[1]:.2f}')
    
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate correlation
    correlation = valid_data['band_gap'].corr(valid_data['formation_energy'])
    print(f"Correlation between Band Gap and Formation Energy: {correlation:.3f}")

## 7. Data by Source

In [ ]:
# Analysis by data source
if 'source' in df.columns:
    source_stats = df.groupby('source').agg({
        'material_id': 'count',
        'band_gap': ['mean', 'std', 'min', 'max'],
        'formation_energy': ['mean', 'std', 'min', 'max']
    }).round(3)
    
    source_stats.columns = ['_'.join(col).strip() for col in source_stats.columns.values]
    source_stats.rename(columns={'material_id_count': 'count'}, inplace=True)
    
    print("Statistics by Data Source:")
    display(source_stats)
    
    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Count by source
    df['source'].value_counts().plot(kind='bar', ax=axes[0])
    axes[0].set_title('Materials by Source')
    axes[0].set_ylabel('Count')
    
    # Band gap by source
    df.boxplot(column='band_gap', by='source', ax=axes[1])
    axes[1].set_title('Band Gap by Source')
    axes[1].set_ylabel('Band Gap (eV)')
    
    # Formation energy by source
    df.boxplot(column='formation_energy', by='source', ax=axes[2])
    axes[2].set_title('Formation Energy by Source')
    axes[2].set_ylabel('Formation Energy (eV/atom)')
    
    plt.suptitle('')  # Remove automatic title
    plt.tight_layout()
    plt.show()

## 8. Simple ML Model (Baseline)

In [ ]:
# Load ML-ready data
ml_file = data_dir / 'ml_ready_data.pkl'
if ml_file.exists():
    with open(ml_file, 'rb') as f:
        ml_data = pickle.load(f)
    
    print(f"ML Data Loaded:")
    print(f"  Samples: {ml_data['n_samples']}")
    print(f"  Features: {ml_data['n_features']}")
    print(f"  Feature names: {ml_data['feature_names']}")
else:
    print("No ML-ready data found. Run data pipeline first!")
    ml_data = None

In [ ]:
# Train simple Random Forest model for Band Gap
if ml_data and ml_data['features'] is not None:
    X = ml_data['features']
    y_bandgap = ml_data['band_gap']
    
    # Remove samples with NaN
    mask = ~np.isnan(X).any(axis=1) & ~np.isnan(y_bandgap)
    X_clean = X[mask]
    y_clean = y_bandgap[mask]
    
    if len(X_clean) > 10:
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_clean, y_clean, test_size=0.2, random_state=42
        )
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train model
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train_scaled, y_train)
        
        # Predictions
        y_pred = rf_model.predict(X_test_scaled)
        
        # Metrics
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"Band Gap Prediction Results:")
        print(f"  MAE: {mae:.3f} eV")
        print(f"  R²: {r2:.3f}")
        
        # Visualization
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        # Predicted vs Actual
        axes[0].scatter(y_test, y_pred, alpha=0.6)
        axes[0].plot([y_test.min(), y_test.max()], 
                    [y_test.min(), y_test.max()], 
                    'r--', lw=2)
        axes[0].set_xlabel('Actual Band Gap (eV)')
        axes[0].set_ylabel('Predicted Band Gap (eV)')
        axes[0].set_title(f'Band Gap Predictions (R² = {r2:.3f})')
        axes[0].grid(True, alpha=0.3)
        
        # Feature importance
        importances = pd.DataFrame({
            'feature': ml_data['feature_names'],
            'importance': rf_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        axes[1].barh(range(len(importances)), importances['importance'].values)
        axes[1].set_yticks(range(len(importances)))
        axes[1].set_yticklabels(importances['feature'].values)
        axes[1].set_xlabel('Feature Importance')
        axes[1].set_title('Random Forest Feature Importances')
        
        plt.tight_layout()
        plt.show()
    else:
        print("Not enough clean samples for training")
else:
    print("No features available for ML model")

## 9. Save Processed Data for Further Analysis

In [ ]:
# Save cleaned dataset for further analysis
if not df.empty:
    # Create analysis directory
    analysis_dir = Path('../data/analysis')
    analysis_dir.mkdir(exist_ok=True)
    
    # Save complete samples only
    if 'is_complete' in df.columns:
        complete_df = df[df['is_complete']]
        output_file = analysis_dir / 'complete_materials.csv'
        complete_df.to_csv(output_file, index=False)
        print(f"Saved {len(complete_df)} complete materials to {output_file}")
    
    # Save summary statistics
    summary_stats = df.describe()
    summary_file = analysis_dir / 'summary_statistics.csv'
    summary_stats.to_csv(summary_file)
    print(f"Saved summary statistics to {summary_file}")

## 10. Conclusions and Next Steps

### Key Findings:
- Document your findings here after running the analysis
- Note any interesting patterns or outliers
- Identify potential features for ML models

### Next Steps:
1. **Feature Engineering**: Create additional features based on domain knowledge
2. **Advanced ML Models**: Try XGBoost, Neural Networks, etc.
3. **Hyperparameter Tuning**: Optimize model parameters
4. **Cross-validation**: Implement proper CV strategies
5. **Ensemble Methods**: Combine multiple models for better predictions

In [ ]:
print("\n" + "="*50)
print("Exploratory Analysis Complete!")
print("="*50)